# Finding prices using LLMs and Langgraph

The objective on this notebook is to find a price from some object from specific URL's, in the notebook example, we will use the [zoom](https://www.zoom.com.br).

For this task, we gonna use Langgraph and Langchain, to divide this task in some particular tasks. 

---

Importing packages:

In [192]:
from dotenv import load_dotenv

from langchain_openai.chat_models import ChatOpenAI

from langchain_community.document_loaders import WebBaseLoader

from langchain_core.messages import (
    AnyMessage, 
    AIMessage, 
    HumanMessage, 
    SystemMessage
)

from langgraph.graph import StateGraph, START, END, MessagesState

from re import search

from os import getenv

from typing import Literal

Configurations:

In [34]:
load_dotenv()

True

In [35]:
OPEN_AI_URL = getenv("OPENAI_URL")
OPEN_AI_MODEL = getenv("MODEL")

In [78]:
RETAIL_URL = "https://www.zoom.com.br"

---

First, we need to define a way to use the retail store url to search some prices!

In [79]:
RETAIL_SUFIX = lambda input: f"{RETAIL_URL}/search?q={input.replace(' ','%20')}"

Now, let's consume the first retail for some respective input. For this, we will use 

In [80]:
INPUT = "ar condicionado"

In [81]:
url_search = RETAIL_SUFIX(INPUT)

In [82]:
url_search

'https://www.zoom.com.br/search?q=ar%20condicionado'

In [84]:
web_loaded = WebBaseLoader(url_search)

In [92]:
[document] = web_loaded.load()

Let's dive in on what is on the document!

In [94]:
document.dict().keys()

/tmp/ipykernel_34540/1084269162.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  document.dict().keys()


dict_keys(['id', 'metadata', 'page_content', 'type'])

In [95]:
document.id

In [96]:
document.metadata

{'source': 'https://www.zoom.com.br/search?q=ar%20condicionado',
 'title': 'Ar condicionado com o menor preço | Zoom ',
 'description': 'Buscando o menor preço para ar condicionado LG, Springer Midea, Gree e mais? No Zoom você compara as melhores ofertas com segurança e ainda aproveita cupons e cashback.',
 'language': 'pt-BR'}

In [101]:
print(document.page_content)

Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.000 

In [99]:
document.type

'Document'

Now, we can use a textual content in a LLM!

But before, let's find a way to delimiter the retails on the search!

In [128]:
content_split_lines = document.page_content.splitlines()

In [134]:
RETAILS = [
    "Casas Bahia",
    "Extra",
    "Magazine Luiza"   
]

In [135]:
content_split_lines

['Ar condicionado com o menor preço | Zoom CategoriasCategoriasCelularesCelularesiPhoneiPhone 16iPhone 15iPhone 14iPhone 13iPhone 12iPhone 11Celulares SamsungGalaxy S25Galaxy S24Galaxy S23Galaxy S22Galaxy A56Galaxy A54Celulares MotorolaMoto GMoto ECelulares XiaomiXiaomi RedmiCelulares RealmeVer mais em celularesTVsTVsTV 32 polegadasTV 43 polegadasTV 50 polegadasTV 55 polegadasTV 65 polegadasTV 4KTV SamsungTV LGVer mais em TVsLavadorasLavadorasLavadora BrastempLavadora ElectroluxLavadora PanasonicLavadora ConsulLava e SecaLava e Seca LGLava e Seca SamsungNotebooksNotebooksNotebook  SamsungNotebook  DellNotebook  LenovoNotebook  AcerNotebook  AsusNotebook  GamerNotebook  i7Notebook  i5MacbookMacbook AirMacbook ProVer mais em notebooksGeladeirasGeladeirasGeladeira ElectroluxGeladeira BrastempGeladeira PanasonicGeladeira ConsulGeladeira InverseGeladeira Frost FreeGeladeira DuplexGeladeira InoxVer mais em geladeirasAr-CondicionadoAr-CondicionadoAr-condicionado 7.500 BTUsAr-condicionado 9.00

In [160]:
pattern = rf'({"|".join(RETAILS)})'

pattern

'(Casas Bahia|Extra|Magazine Luiza)'

In [161]:
lines_with_retails = [content.strip() for content in content_split_lines if search(pattern, content)]

In [162]:
lines_with_retails

['6% de volta na loja todaCompare entre 14 lojasIndicaAr-Condicionado Split Hi Wall LG Dual Inverter Voice 12000 BTUs Frio Inverter S4-Q12JA314.7 (346)Menor preço via ExtraR$ 2.374,05até 10x de R$ 249,90',
 '0.5% de volta na loja todaCompare entre 7 lojasIndicaAr-Condicionado Split Hi Wall Midea Xtreme Save Connect 12000 BTUs Frio Inverter 42AGVCI12M5 / 38AGVCI12M54.6 (268)Menor preço via ExtraR$ 2.203,20até 10x de R$ 244,80',
 '0.5% de volta na loja todaCompare entre 6 lojasAr-Condicionado Split Hi Wall Springer Midea AI Ecomaster 12000 BTUs Frio Inverter 42EZVCA12M5 / 38EZVCA12M5Menor preço via Magazine LuizaR$ 2.186,10até 12x de R$ 202,42',
 '0.5% de volta na loja todaCompare entre 15 lojasAr-Condicionado Split Hi Wall Agratto 9000 BTUs Inverter Controle Remoto Frio Liv Top LCST9FI / LCST9FE4.5 (35)Menor preço via Fast ShopR$ 1.639,00 0.5% de volta na loja todaCompare entre 11 lojasIndicaAr-Condicionado Split Hi Wall Samsung Wind Free Connect 12000 BTUs Frio Inverter AR12CVFAMWK4.6 

Now, let's send this to a LLM!

In [163]:
chat = ChatOpenAI(base_url=OPEN_AI_URL, model=OPEN_AI_MODEL)

In [182]:
breakline = lambda: '\n'

messages: list[AnyMessage] = [
    SystemMessage(content="Você é um assistente especializado em ler uma série de ofertas com diferentes preços e trazer a melhor oferta e a sua respectiva loja!", name="System"),
    HumanMessage(content=f"Me dê a melhor oferta a partir das seguintes opções:\n{breakline().join(lines_with_retails)}\nNão esqueça de me falar onde comprar", name="Author")
]

In [185]:
answer = chat.invoke(messages)

In [186]:
print(answer.content)

Vou comparar as opções para você!

**Melhor Opção:** Ar-Condicionado Split Hi Wall LG Dual Inverter Voice 12000 BTUs com 6% de volta na loja toda, preço de R$2.374,05, até 10x de R$249,90.

**Comparação dos Preços:**

| Loja | Preço |
| --- | --- |
| Extra | R$2.203,20 (até 10x de R$244,80) |
| Magazine Luiza | R$2.360,07 (até 10x de R$268,19) |
| Casas Bahia | R$1.579,00 (até 10x de R$175,44) |

**Principais Fabricantes:**

| Fabricante | Produtos Indicados por Especialistas | Número de Produtos |
| --- | --- | --- |
| LG | Split Hi Wall | 46 |
| Springer Midea | Split Hi Wall | 30 |
| Samsung | Split Cassete/Split Piso/Teto | 25 |

**Consumo Mensal de Energia:**
O consumo mensal do ar-condicionado é aproximadamente de R$74,34 por mês para um modelo com capacidade de refrigeração de 12.000 BTUs.

Para encontrar essa opção, você pode visitar as lojas seguintes:

* Extra
* Magazine Luiza
* Casas Bahia

Lembre-se de que o preço e a disponibilidade podem ser diferentes quando comprado dir

All right! Now we understand how to use the ``WebBaseLoader`` to load websites and take text from it!

Our objective is to create a simple graph to automatize this process!